In [1]:
# import packages
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import requests
from bs4 import BeautifulSoup
import time
import lxml
import re
import string

# Parsing XML Files from ProPublica

## Background

Within San Diego County, there is an increasing homelessness crisis. The funding to help take the next steps towards ending this crisis comes from the government, county funding, city funding, and funding from nonprofit organizations. Each nonprofit organization has a service to aid the homeless, yet a lot of money has been put towards the issue and there hasn't been much progress.

In order to fully understand how much money is put towards homelessness within San Diego County, we are creating a dataset that includes all four sources of funding. The overarching goal of this project and dataset is to see how much money is put towards homelessness services within San Diego County with the intent that eventually we can get a better idea of how much money it would take to bring a single person out of homelessness.

For this branch of this homelessness project, compiling the philanthropic data will help identify how much money is put towards homelessness services in San Diego and the sources of funding for each nonprofit organization.

The *Philanthropic Homelessness Data Pull* is a script that pulls data from ProPublica, a tax site that uploads tax returns for nonprofit organizations, searchable through the organization's unique EIN. Compiling this data together, we can get the total philanthropic dollars for homelessness within recent years given that we have the full EIN list.

### Maintaining Data

We have code that pulls the 990 XML files for each provider from ProPublica. We have a list of provider’s EIN numbers that our script pulls from. This list is based on the providers that we have identified from the city contracts and county contracts. There are organizations that perform homelessness services but do not take government dollars. Some of these organizations are not contained and should be added once they are identified.

It is important to note that the parser code we have does it through the 990 XML files under a tax year on ProPublica, but some of the earlier years (like 2014 or 2015) for some EINs don’t have an XML file, so there are some years that has data that’s not collected via the parser code. To correct this, the manual-pull dataset was used to fill in the data that was missing and was manually checked for any data that needed to be added. You can just use the most up-to-date dataset and run through the code function to do this.

### Code Upkeep

The Master EIN List on Google Sheets should be continuously updated with new nonprofits found and updated within the jupyter or GitHub folder. Be sure you're using the updated list for the code.

The most recent dataset should be used under the 'Getting Manual Pull Rows that aren't included in the Scraped Data' section of the code. This is important because there isn't an XML file for some of the older years and that data would be missed without the merging of the dataset within that code.

The csv files can be overwritten as it's moreso a way to keep progress without recalling the grabber() function or messing with the data, but keep the final 'Phil Dataset.csv', 'Phil Dataset - Analysis.csv', and the csv sheets that were pulled from Google Sheets.

### Methods

`scrape_website(url, attempts=0)`
- Takes a URL and returns the response object if successful. Retries three times on failure

`info_grabber(fp, url=True)`
- Takes the URL/filepath of an XML file. The `url` parameter indicates whether the given filepath is a URL or local
- Returns a dictionary of the row information for section VIII (along with a couple other columns)
- Older (prior to 2013) XML files have different tags and thus can't be read. Results in a row of all zeroes and `'Not Found'` in the `ba` column
- Of successfully scraped columns, has ~1.5% error rate for rows (0.13% of cells are inaccurate), only in the `Fundraising Events` column due to inconsistent labeling in XMLs
    - This error rate is lower than the human error rate
- In 2013, they had a different way of tagging zip codes, so all 2013 zips will be 0

`contact_grabber(fp, url=True)`
- Returns a dataframe of each contact in the 990 form, according to section VII
- Incomplete and edge cases aren't tested fully, as we didn't need the function.

`get_xmls(ein)`
- Takes the EIN of a company and returns a list of the URLs for each XML file that is linked on the ProPublica page for that given company

`grabber(eins, verbose=False, clean=False)`
- Takes a list of EINs and uses `get_xmls()` to find each XML file, then uses `info_grabber()` to scrape each XML for the relevant information.
- Returns a pandas DataFrame, doesn't automatically save as csv.
- Takes around 22 minutes on my laptop for 71 businesses
- The `clean` parameter automatically cleans the dataframe by removing empty rows

`find_errors(info)`
- Data columns are supposed to add up to the total, if it doesn't this method will find it
- Takes a dataframe of 990 rows from `grabber` and returns a tuple of incides that didn't properly grab all of the data
    - Preferably indiced on `EIN_YEAR`
    
##### Helper Methods
`standardize_text(text)`
- Removes any unwanted punction and turns inputted text to upper-case to standardize it. Used for the dataframes.
    
`replace_corrected_data_and_standardize(df1, df2, concat=False)`
- Used for replacing the information in 'df2' that appears in 'df1' so that the new corrected information in 'df2' is present in the full dataframe 'df1'. 
    - This is used after the double filtering of the errors to replace the incorrect information with the corrected information.
- Standardizes the text in both dataframes for accurate replacement.
- The concat input can be set to true to standardize and combine dataframes to add new information from 'df2' instead of just replacing. 
    - This was added to add any missing information that was on the 990 manual pull but not the generated dataset.
    
`distribute_values_fiscal_year(row)`
- Expand fiscal years into months based on 'Tax End Date' column and divides columns with dollar values by the number of months
    - apply to df by doing expanded_data_fiscal_year = df.apply(distribute_values_fiscal_year, axis=1)
    - expanded_data_fiscal_year =  pd.DataFrame([item for sublist in expanded_data_fiscal_year for item in sublist])


#### XML Parsing methods

#### `scrape_website`

In [62]:
def scrape_website(url, attempts=0):
    '''
    Takes a URL and returns the response object if successful. Retries three times on failure
    '''
    # Send GET request to the website
    response = requests.get(url)
    time.sleep(1)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        if attempts > 0:
            print('Success')
        return response
    elif attempts < 3:
        print("Error:", response.status_code, "Trying again.")
        time.sleep(1)
        return scrape_website(url, attempts + 1)
    else:
        # Handle the request error
        print("Error: ", response.status_code, 'Failed on:', url)
        return None

#### `info_grabber`

In [7]:
def info_grabber(fp, url=True, errors=False):
    ''' 
    - Takes the URL/filepath of an XML file. The url parameter indicates whether the given filepath is a URL or local
    - Returns a dictionary of the row information for section VIII (along with a couple other columns)
    - Older (prior to 2013) XML files have different tags and thus can't be read. Results in a row of all zeroes and 'Not Found' in the ba column
    - Of successfully scraped columns, has ~1.5% error rate for rows (0.13% of cells are inaccurate), only in the Fundraising Events column due to inconsistent labeling in XMLs
    - This error rate is lower than the human error rate
    - In 2013, they had a different way of tagging zip codes, so all 2013 zips will be 0
    '''
    # Use scrape_website(url) to get XML or read file
    if url:
        xml = scrape_website(fp)
        xml.encoding = 'UTF-8'
        xml = xml.text.strip()
    else:
        try:
            with open(fp, 'r') as file:
                xml = file.read()
        except FileNotFoundError:
            print(f"File not found: {fp}")
            return dict()
    if xml is None:
        return dict()
    
    soup = BeautifulSoup(xml, 'xml')
    
    # Assigning dict values to tags scraped from pulled XML file
    row = dict()
    row['ba'] = soup.find('BusinessName')
    row['EIN'] = soup.find('EIN')
    row['Tax Year'] = soup.find('TaxPeriodBeginDt')
    row['Tax End Date'] = soup.find('TaxPeriodEndDt')
    row['Location (Zipcode)'] = soup.find('USAddress')
    row['Federate Campaigns'] = soup.find('FederatedCampaignsAmt')
    row['Membership Dues'] = soup.find('MembershipDuesAmt')
    row['Fundraising Events'] = soup.find('ContriRptFundraisingEventAmt')
    
    row['Related Organizations'] = soup.find('RelatedOrganizationsAmt')
    row['Government Grants'] = soup.find('GovernmentGrantsAmt')
    row['All Other Contributions'] = soup.find('AllOtherContributionsAmt')
    row['Noncash Contributions'] = soup.find('NoncashContributionsAmt')
    row['Total'] = soup.find('TotalContributionsAmt')

    # tries different tag
    if row['Location (Zipcode)'] is not None:
        row['Location (Zipcode)'] = row['Location (Zipcode)'].find('ZIPCd')
    else:
        row['Location (Zipcode)'] = None

    # organizes dict properly
    row = dict(map(lambda item: (item[0], item[1].text if item[1] is not None else None), row.items()))

    if row['ba'] is not None:
        row['ba'] = row['ba'].strip()
    else:
        row['ba'] = 'Not found'
    # correcting 'Tax Year' format
    if row['Tax Year'] is not None:
        row['Tax Year'] = pd.Timestamp(row['Tax Year']).year
    else:
        row['Tax Year'] = np.nan
    # correcting 'Tax End Date' format
    if row['Tax End Date'] is not None:
        row['Tax End Date'] = pd.to_datetime(row['Tax End Date']).strftime("%m-%d-%Y")
    else:
        row['Tax End Date'] = np.nan
    
    # use errors=True on EINs that returned incorrect sums on the first grabber() call
    # calculates sum with new fundraising tag to see if it fixes the errors
    if errors:
        sumKeys = ['Federate Campaigns', 'Membership Dues', 'Related Organizations', 'Government Grants', 'All Other Contributions']
        sumAmt = sum(float(row[key]) for key in sumKeys if key in row and row[key] is not None)
        fundAmt_element = soup.find('FundraisingAmt')
        fundAmt = int(fundAmt_element.text) if fundAmt_element is not None else 0
        sumAmt += fundAmt

        # See if the other fundraising tag fixes error and if it does, then replace it
        if sumAmt is None or row['Total'] is None:
            pass
        elif int(sumAmt) == int(row['Total']):
            row['Fundraising Events'] = fundAmt
        else:
            pass
    # cleaning format
    row = dict(map(lambda item: (item[0], item[1] if item[1] is not None else 0), row.items()))

    return row


#### `contact_grabber`

In [8]:
def contact_grabber(fp, url=True):
    '''
    Returns a dataframe of each contact in the 990 form, according to section VII
    Incomplete and edge cases aren't tested fully, as we didn't need the function.

    :param fp: filepath of the xml file
    :param url: Whether the fp is a url or not. Default = True
    '''

    # If 'url' is True, assume 'fp' is a URL and call a function 'scrape_website' to retrieve the XML data.
    # If 'url' is False, assume 'fp' is a file path and try to fetch the file contents using 'requests'.
    if url:
        xml = scrape_website(fp)
    else:
        xml = requests.get(fp)

    # If 'xml' is None, return an empty dictionary and exit the function.
    if xml is None:
        return dict()

    # Set the encoding of 'xml' to 'UTF-8' and strip any leading/trailing whitespace.
    xml.encoding = 'UTF-8'
    xml = xml.text.strip()

    # Parse the 'xml' data using BeautifulSoup with an 'xml' parser.
    soup = BeautifulSoup(xml, 'xml')

    # Find all elements with the tag 'Form990PartVIISectionAGrp' in the parsed XML.
    people = soup.find_all('Form990PartVIISectionAGrp')

    # Create lists 'names' and 'titles' containing text from 'PersonNm' and 'TitleTxt' elements, respectively,
    # for each 'Form990PartVIISectionAGrp' element that has these sub-elements.
    names = [x.find('PersonNm').text for x in people if x.find('PersonNm') is not None]
    titles = [x.find('TitleTxt').text for x in people if x.find('TitleTxt') is not None]

    # Try to find the organization's name by looking for the 'BusinessName' element.
    # If not found, set 'organization' to 'Not found' for each contact.
    try:
        organization = [soup.find('BusinessName').text.strip()] * len(names)
    except AttributeError:
        organization = ['Not found'] * len(names)

    # Try to find the year by parsing the 'TaxPeriodBeginDt' element and extracting the year.
    # If not found, set 'year' to 0 for each contact.
    try:
        year = [pd.Timestamp(soup.find('TaxPeriodBeginDt').text).year] * len(names)
    except AttributeError:
        year = [0] * len(names)

    # Return a Pandas DataFrame containing the extracted data: organization, year, names, and titles.
    return pd.DataFrame({'organization': organization, 'year': year, 'names': names, 'titles': titles})


#### `get_xmls`

In [9]:
def get_xmls(ein):
    '''
    Return a list of xml urls from ProRepublica based on an organization's EIN

    :param ein: Employer Identification number as String or Int
    '''

    # Create a URL using the given 'ein' to fetch information from ProPublica's website.
    url = f'https://projects.propublica.org/nonprofits/organizations/{ein}'

    # Call a function 'scrape_website' to retrieve the HTML content of the URL.
    html = scrape_website(url)

    # If 'html' is None (indicating an error or empty response), return an empty list.
    if html is None:
        return []

    # Parse the HTML content using BeautifulSoup.
    soup = BeautifulSoup(html.text)

    # Find all anchor elements ('a') with the class 'action xml' in the parsed HTML.
    links = soup.find_all('a', class_='action xml')

    # Define the base URL for constructing the complete XML URLs.
    base_url = 'https://projects.propublica.org'

    # Extract XML URLs for 990 forms from the found anchor elements.
    # The condition 'if x.text == '990'' filters for links with the text '990'.
    urls = [base_url + x.get('href') for x in links if x.text == '990']

    # Alternatively, you can achieve the same result using a regular expression to filter links.
    # This section replaces the previous one.
    urls = [x for x in soup.find_all(class_='action xml') if re.search(r'990\b', x.text)]
    urls = [base_url + x.get('href') if x.name == 'a' else base_url + x.select_one('select.action.xml option[data-href]').get('data-href') for x in urls]

    # Return the list of extracted XML URLs.
    return urls

#### For errors

*use `grabber(errors = True)` to correct errors after calling find_errors(df)*

#### `grabber`

In [63]:
def grabber(eins, verbose=False, clean=False, errors=False):
    '''
    - Takes a list of EINs and uses get_xmls() to find each XML file, then uses info_grabber() to scrape each XML for the relevant information.
    - Returns a pandas DataFrame, doesn't automatically save as csv.
    - Takes around 22 minutes on my laptop for 71 businesses
    - The clean parameter automatically cleans the dataframe by removing empty rows
    '''

    # Check if 'eins' is of type int, and if so, convert it to a list containing that int.
    if type(eins) == int:
        eins = [eins]

    # Create an empty list 'data' to store the scraped data.
    data = []

    # Initialize an 'overall_index' variable to keep track of the overall progress.
    overall_index = 0

    # Loop through each EIN in the 'eins' list.
    for index, ein in enumerate(eins):
        print(str(index) + (' / ') + str(len(eins)))

        # Call the 'get_xmls' function to retrieve XML URLs for the current EIN.
        xmls = get_xmls(ein)

        # Loop through each XML URL.
        for xml in xmls:
            if verbose:
                print(f'{overall_index} {xml}')

            # If 'errors' is True, call the grabber() function with 'errors=True'.
            # This is used when double filtering errors already found by calling 'find_errors(df)'.
            if errors:
                data.append(info_grabber(xml, errors=True))
            else:
                data.append(info_grabber(xml))

            # Increment the 'overall_index' to keep track of progress across all EINs.
            overall_index += 1

    # Print the total number of EINs processed.
    print(str(len(eins)) + (' / ') + str(len(eins)))

    # Create a DataFrame 'info' containing the collected data.
    info = pd.DataFrame(data)

    # If 'clean' is False, return the DataFrame 'info' as is.
    if not clean:
        return info

    # If 'clean' is True, perform data cleaning operations on the DataFrame 'info'.
    # Remove rows with missing values (NaN), convert certain columns to integers, and create an 'EIN_YEAR' column.
    no_error = info.dropna().reset_index(drop=True)
    no_error.loc[:, 'EIN':'Total'] = no_error.loc[:, 'EIN':'Total'].astype(int)
    no_error.loc[:, 'EIN_YEAR'] = no_error['EIN'].astype(str) + '_' + no_error['Tax Year'].astype(str)
    no_error = no_error.set_index('EIN_YEAR')

    # Return the cleaned DataFrame 'no_error'.
    return no_error

#### `find_errors`

In [11]:
def find_errors(info):
    '''
    Takes a dataframe of 990 rows and returns a tuple of the indices where it failed to grab the data properly

    :param info: The dataframe of 990 rows
    '''
    # Calculate a Boolean Series 'accurate' by checking if the sum of certain columns equals the 'Total' column.
    accurate = info[['Federate Campaigns', 'Membership Dues', 'Fundraising Events', 'Related Organizations', 'Government Grants', 'All Other Contributions']].sum(axis=1) == info['Total']
    # Return a tuple containing the indices (row numbers) where 'accurate' is False.
    return tuple(accurate[~accurate].index)

#### Standardize text in dataframe

#### `standardize_text`

In [12]:
def standardize_text(text):
    '''
    Removes any unwanted punction and turns inputted text to upper-case to standardize it. Used for the dataframes.
    '''
    return re.sub(r'[{}\'‘’]'.format(string.punctuation), '', text).upper().strip().replace('\n',' ')

#### Update dataframe with corrections from new dataframe

Use after `find_errors(df)` and output from `grabber(errors=True)`

#### `replace_corrected_data_and_standardize`

In [13]:
def replace_corrected_data_and_standardize(df1, df2, concat=False):
    '''
    Expand fiscal years into months based on 'Tax End Date' column and divides columns with dollar values by the number of months
    - apply to df by doing expanded_data_fiscal_year = df.apply(distribute_values_fiscal_year, axis=1)
      expanded_data_fiscal_year = pd.DataFrame([item for sublist in expanded_data_fiscal_year for item in sublist])
    '''

    # Define a list of columns that should be standardized (excluding 'Tax End Date').
    to_std = ['Organization Name', 'EIN', 'Tax Year', 'Location (Zipcode)', 'Federate Campaigns', 'Membership Dues',
              'Fundraising Events', 'Related Organizations', 'Government Grants', 'All Other Contributions',
              'Noncash Contributions', 'Total']

    # Reset the index of both input DataFrames 'df1' and 'df2'.
    df1 = df1.reset_index()
    df2 = df2.reset_index()

    # Standardize text values in the selected columns for both 'df1' and 'df2'.
    for col in to_std:
        if col != 'Tax End Date':
            df1[col] = df1[col].apply(lambda x: standardize_text(x) if isinstance(x, str) else x)
            df2[col] = df2[col].apply(lambda x: standardize_text(x) if isinstance(x, str) else x)

    # Replace errors in 'df1' with corrected information from 'df2' based on matching EIN_YEAR values.
    for index_value, corrected_row in df2.iterrows():
        ein_year = corrected_row['EIN_YEAR']
        matching_rows = df1['EIN_YEAR'] == ein_year
        for column in df1.columns:
            df1.loc[matching_rows, column] = corrected_row[column]

    # If 'concat' is False, assign 'df1' to 'combined_df'. Otherwise, concatenate 'df1' and 'df2'.
    if not concat:
        combined_df = df1
    else:
        combined_df = pd.concat([df1, df2])

    # Calculate a new column 'Philanthropic_Total' by subtracting 'Government Grants' from 'Total'.
    combined_df['Philanthropic_Total'] = combined_df['Total'] - combined_df['Government Grants']

    # Return the combined DataFrame 'combined_df'.
    return combined_df

#### Expand by months

#### `distribute_values_fiscal_year`

In [14]:
def distribute_values_fiscal_year(row):
    # Calculate the start and end dates of the fiscal year based on the 'Tax End Date' column.
    fiscal_year_start_date = pd.Timestamp(year=row['Tax End Date'].year - 1, month=row['Tax End Date'].month, day=row['Tax End Date'].day) + pd.DateOffset(days=1)
    fiscal_year_end_date = fiscal_year_start_date + pd.DateOffset(years=1, days=-1)

    # Calculate the number of months in the fiscal year.
    num_months = (fiscal_year_end_date.year - fiscal_year_start_date.year) * 12 + fiscal_year_end_date.month - fiscal_year_start_date.month + 1

    # Calculate the values to distribute (including the last two columns) by dividing by the number of months.
    values_to_distribute = row[7:] / num_months

    # Generate a date range for each month within the fiscal year.
    date_range = pd.date_range(start=fiscal_year_start_date, end=fiscal_year_end_date, freq='M')

    # Create a list 'new_rows' to store the new rows for each month.
    new_rows = []

    # Iterate over the date range and create a new row for each month.
    for date in date_range:
        new_row = row.copy()
        
        # Update the 'Tax End Date' column with the current date in 'MM-DD-YYYY' format.
        new_row['Tax End Date'] = date.strftime('%m-%d-%Y')
        
        # Update all columns from the 7th column onwards with the values to distribute.
        new_row[7:] = values_to_distribute
        
        # Append the new row to the 'new_rows' list.
        new_rows.append(new_row)

    # Return the list of new rows, effectively expanding the fiscal year data into monthly data.
    return new_rows


### Using the `grabber()` method

`grabber()` takes a list of EINs. This was curated from the old "990 Manual Pull" spreadsheet by pasting and splitting the entire `EIN` column and putting it into a set.

In [11]:
all_eins = '''113723093
113723093
204374795
237161267
237279074
237334012
261712580
261712580
263405689
270447059
270447059
270865318
270865318
272917644
273390797
330006089
330008269
330008269
330122462
330210280
330215585
330215585
330217339
330317950
330433314
330492304
330492304
330496092
330497515
330553621
330553621
330553621
330602842
330602842
330618893
330623634
330902617
330902617
331029843
331029843
331146733
331146733
465055513
465055513
550806460
562613191
571162424
571162424
571162424
571214920
571214920
571214920
821946283
822363154
931008369
941676390
942358632
942358632
951644024
951648184
951869806
951874073
951944230
951945256
952039198
952039198
952111196
952157559
952157559
952213995
952422704
952422704
952457354
952457354
952648050
952653869
952693142
952693142
952794253
952794253
952794253
952833205
952833205
952833205
952850121
952877102
952880426
952880426
952949636
953031682
953138268
953138268
953140767
953140767
953244085
953244085
953248148
953248148
953302967
953302967
953315571
953368020
953497926
953649525
953649525
953750738
953782164
953782164
953798088
953837714
953837714
953950196
953950196
953950196
956379598
204374795
311750839
330553704
330498557
880279721
330352148
330352148
951729734
237332048'''

In [12]:
eins = list(set(all_eins.split()))

<b>Warning</b>: `grabber()` takes a long time. The following cell will take about 20 minutes.

# 990 - Manual Pull Testing

In [13]:
info990 = grabber(eins, verbose=True)

0 / 83
0 https://projects.propublica.org/nonprofits/download-xml?object_id=202341239349300104
1 https://projects.propublica.org/nonprofits/download-xml?object_id=202200889349300050
2 https://projects.propublica.org/nonprofits/download-xml?object_id=202110429349301336
3 https://projects.propublica.org/nonprofits/download-xml?object_id=202030799349300613
4 https://projects.propublica.org/nonprofits/download-xml?object_id=201900109349301975
5 https://projects.propublica.org/nonprofits/download-xml?object_id=201733489349300023
6 https://projects.propublica.org/nonprofits/download-xml?object_id=201740139349301144
7 https://projects.propublica.org/nonprofits/download-xml?object_id=201600409349301435
1 / 83
8 https://projects.propublica.org/nonprofits/download-xml?object_id=202203159349307235
9 https://projects.propublica.org/nonprofits/download-xml?object_id=202113199349316051
2 / 83
10 https://projects.propublica.org/nonprofits/download-xml?object_id=202301359349318435
11 https://projects.p

93 https://projects.propublica.org/nonprofits/download-xml?object_id=201521339349305167
94 https://projects.propublica.org/nonprofits/download-xml?object_id=201440279349300889
95 https://projects.propublica.org/nonprofits/download-xml?object_id=201330919349300318
96 https://projects.propublica.org/nonprofits/download-xml?object_id=201111259349301576
11 / 83
97 https://projects.propublica.org/nonprofits/download-xml?object_id=202341119349301419
98 https://projects.propublica.org/nonprofits/download-xml?object_id=202211239349301641
99 https://projects.propublica.org/nonprofits/download-xml?object_id=202032749349300628
12 / 83
100 https://projects.propublica.org/nonprofits/download-xml?object_id=202202279349302275
101 https://projects.propublica.org/nonprofits/download-xml?object_id=202112249349302051
102 https://projects.propublica.org/nonprofits/download-xml?object_id=202022309349302537
103 https://projects.propublica.org/nonprofits/download-xml?object_id=201922219349300012
104 https://

185 https://projects.propublica.org/nonprofits/download-xml?object_id=201242559349300804
186 https://projects.propublica.org/nonprofits/download-xml?object_id=201143129349300944
21 / 83
187 https://projects.propublica.org/nonprofits/download-xml?object_id=202213139349301966
188 https://projects.propublica.org/nonprofits/download-xml?object_id=202103139349304345
189 https://projects.propublica.org/nonprofits/download-xml?object_id=202043189349313114
190 https://projects.propublica.org/nonprofits/download-xml?object_id=202002479349300135
22 / 83
191 https://projects.propublica.org/nonprofits/download-xml?object_id=202201089349301025
192 https://projects.propublica.org/nonprofits/download-xml?object_id=202121269349302937
193 https://projects.propublica.org/nonprofits/download-xml?object_id=202001639349301315
194 https://projects.propublica.org/nonprofits/download-xml?object_id=201901349349309420
195 https://projects.propublica.org/nonprofits/download-xml?object_id=201831359349307543
196 h

277 https://projects.propublica.org/nonprofits/download-xml?object_id=201423179349306362
278 https://projects.propublica.org/nonprofits/download-xml?object_id=201333189349306243
279 https://projects.propublica.org/nonprofits/download-xml?object_id=201203149349301865
280 https://projects.propublica.org/nonprofits/download-xml?object_id=201133149349301708
32 / 83
281 https://projects.propublica.org/nonprofits/download-xml?object_id=201911349349307891
282 https://projects.propublica.org/nonprofits/download-xml?object_id=201812999349301026
283 https://projects.propublica.org/nonprofits/download-xml?object_id=201722989349301462
284 https://projects.propublica.org/nonprofits/download-xml?object_id=201622259349303997
285 https://projects.propublica.org/nonprofits/download-xml?object_id=201503169349303225
33 / 83
286 https://projects.propublica.org/nonprofits/download-xml?object_id=202331189349300303
287 https://projects.propublica.org/nonprofits/download-xml?object_id=202221319349303917
288 h

369 https://projects.propublica.org/nonprofits/download-xml?object_id=201910689349300231
370 https://projects.propublica.org/nonprofits/download-xml?object_id=201831209349301578
371 https://projects.propublica.org/nonprofits/download-xml?object_id=201701329349306405
372 https://projects.propublica.org/nonprofits/download-xml?object_id=201741329349306329
373 https://projects.propublica.org/nonprofits/download-xml?object_id=201510459349300616
374 https://projects.propublica.org/nonprofits/download-xml?object_id=201410439349301271
375 https://projects.propublica.org/nonprofits/download-xml?object_id=201331979349300833
376 https://projects.propublica.org/nonprofits/download-xml?object_id=201103559349300330
377 https://projects.propublica.org/nonprofits/download-xml?object_id=201110189349300306
42 / 83
378 https://projects.propublica.org/nonprofits/download-xml?object_id=202103419349300605
379 https://projects.propublica.org/nonprofits/download-xml?object_id=202003439349300105
380 https://p

53 / 83
461 https://projects.propublica.org/nonprofits/download-xml?object_id=202222719349301757
462 https://projects.propublica.org/nonprofits/download-xml?object_id=202201369349316620
463 https://projects.propublica.org/nonprofits/download-xml?object_id=202131319349302313
54 / 83
464 https://projects.propublica.org/nonprofits/download-xml?object_id=202201309349304960
465 https://projects.propublica.org/nonprofits/download-xml?object_id=202121349349305352
466 https://projects.propublica.org/nonprofits/download-xml?object_id=202001399349300845
467 https://projects.propublica.org/nonprofits/download-xml?object_id=201932949349300228
468 https://projects.propublica.org/nonprofits/download-xml?object_id=201821309349303792
469 https://projects.propublica.org/nonprofits/download-xml?object_id=201643549349300649
470 https://projects.propublica.org/nonprofits/download-xml?object_id=201523579349300322
471 https://projects.propublica.org/nonprofits/download-xml?object_id=201511149349300116
472 h

553 https://projects.propublica.org/nonprofits/download-xml?object_id=201600439349303570
554 https://projects.propublica.org/nonprofits/download-xml?object_id=201521339349305057
555 https://projects.propublica.org/nonprofits/download-xml?object_id=201400859349300600
556 https://projects.propublica.org/nonprofits/download-xml?object_id=201300619349300505
557 https://projects.propublica.org/nonprofits/download-xml?object_id=201240759349300729
558 https://projects.propublica.org/nonprofits/download-xml?object_id=201130429349300113
64 / 83
559 https://projects.propublica.org/nonprofits/download-xml?object_id=202310309349300871
560 https://projects.propublica.org/nonprofits/download-xml?object_id=202220269349301352
561 https://projects.propublica.org/nonprofits/download-xml?object_id=202140409349301559
562 https://projects.propublica.org/nonprofits/download-xml?object_id=202010289349300421
563 https://projects.propublica.org/nonprofits/download-xml?object_id=201940299349300829
564 https://p

645 https://projects.propublica.org/nonprofits/download-xml?object_id=202141759349300919
646 https://projects.propublica.org/nonprofits/download-xml?object_id=202012209349300431
647 https://projects.propublica.org/nonprofits/download-xml?object_id=201912209349300341
648 https://projects.propublica.org/nonprofits/download-xml?object_id=201842199349301264
649 https://projects.propublica.org/nonprofits/download-xml?object_id=201702269349301270
650 https://projects.propublica.org/nonprofits/download-xml?object_id=201622239349302537
651 https://projects.propublica.org/nonprofits/download-xml?object_id=201512259349303656
652 https://projects.propublica.org/nonprofits/download-xml?object_id=201432259349300643
653 https://projects.propublica.org/nonprofits/download-xml?object_id=201312269349303311
654 https://projects.propublica.org/nonprofits/download-xml?object_id=201222279349304012
655 https://projects.propublica.org/nonprofits/download-xml?object_id=201112289349300891
73 / 83
656 https://p

737 https://projects.propublica.org/nonprofits/download-xml?object_id=201920809349300542
738 https://projects.propublica.org/nonprofits/download-xml?object_id=201810689349300446
739 https://projects.propublica.org/nonprofits/download-xml?object_id=201701329349302285
740 https://projects.propublica.org/nonprofits/download-xml?object_id=201631339349302733
741 https://projects.propublica.org/nonprofits/download-xml?object_id=201500759349301170
742 https://projects.propublica.org/nonprofits/download-xml?object_id=201431139349300913
743 https://projects.propublica.org/nonprofits/download-xml?object_id=201320739349301012
744 https://projects.propublica.org/nonprofits/download-xml?object_id=201201179349300315
745 https://projects.propublica.org/nonprofits/download-xml?object_id=201131329349301888
81 / 83
746 https://projects.propublica.org/nonprofits/download-xml?object_id=202300989349300325
747 https://projects.propublica.org/nonprofits/download-xml?object_id=202240419349300834
748 https://p

In [14]:
info990

,ba,EIN,Tax Year,Tax End Date,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total
0,North County Lifeline Inc\ndba Lifeline Commun...,952794253,2021.0,06-30-2022,92084,0,0,0,0,13405575,451714,223226,13857289
1,North County Lifeline Inc\ndba Lifeline Commun...,952794253,2020.0,06-30-2021,92056,0,0,0,0,13098737,679469,189639,13778206
2,North County Lifeline Inc,952794253,2019.0,06-30-2020,92056,0,0,0,0,12446965,392396,251569,12839361
3,North County Lifeline Inc,952794253,2018.0,06-30-2019,92056,0,0,0,0,10496439,353245,235480,10849684
4,North County Lifeline Inc,952794253,2017.0,06-30-2018,92056,0,0,0,0,9738384,287018,207107,10025402
...,...,...,...,...,...,...,...,...,...,...,...,...,...
756,THE SAN DIEGO LESBIANGAY BISEXUAL\nTRANSGENDER...,237332048,2017.0,06-30-2018,92163,0,0,0,0,2618909,4786412,0,7405321
757,THE SAN DIEGO LESBIANGAY BISEXUAL\nTRANSGENDER...,237332048,2016.0,06-30-2017,92163,0,0,0,0,2674347,4105894,0,6780241
758,THE SAN DIEGO LESBIANGAY BISEXUAL\nTRANSGENDER...,237332048,2015.0,06-30-2016,92163,0,0,0,0,2265389,2588997,0,4854386
759,THE SAN DIEGO LESBIANGAY BISEXUAL\nTRANSGENDER...,237332048,2014.0,06-30-2015,92163,0,0,0,0,1807212,2867228,0,4674440


### Checking the missing cells

The following two cells should both be 1.0.

The first checks the conditional probability that if the name is not found in a row, the tax year wasn't found.

The second checks the inverse conditional probability. Sometimes, if the request attempt tried and failed three times, the row will be entirely empty, where the name is `''` and not `'Not Found'`

In [16]:
# Every time the name isn't found, nothing is found
info990.loc[info990['ba'] == 'Not found']['Tax Year'].isna().mean()

1.0

In [17]:
# And every time the tax year isn't found the name isn't found
(info990[['ba', 'Tax Year']].loc[info990['Tax Year'].isna()]['ba'] == 'Not found').mean()

1.0

This cell indicates how many XML files failed to be accessed

In [18]:
info990['ba'].isna().sum()

0

If the following cell returns 0.0, that indicates that every time the tax year is missing, every single financial cell is zero. All of these rows can be dropped and no information is lost.

In [19]:
info990.loc[info990['Tax Year'].isna(), 'Federate Campaigns':].sum(axis=1).sum()

0

### Cleaning the raw information

Assuming the prior three checks are all adequate, we can drop rows with any nan values.

In [20]:
no_error = info990.dropna().reset_index(drop=True)
int_columns = no_error.columns.difference(['ba','Tax End Date'])  # Skip the 'Tax End Date' column
no_error[int_columns] = no_error[int_columns].astype(int)
# create EIN_YEAR column and set it as index
no_error['EIN_YEAR'] = no_error['EIN'].astype(str) + '_' + no_error['Tax Year'].astype(str)
no_error = no_error.set_index('EIN_YEAR')

### Export here!

<b>REMEMBER TO EXPORT</b>

`no_error` is the cleaned dataframe

`info990` is the raw dataframe with missing rows

In [21]:
no_error.to_csv('no_errors.csv')

In [22]:
info990.to_csv('full_table.csv')

The following line appends the rows to the bottom of the existing dataframe titled `'no_errors.csv'` instead.

It creates a new CSV and it is recommended to rename it back to `no_errors.csv` in the case it is exported successfully

In [23]:
extended = pd.concat([pd.read_csv('no_errors.csv'), no_error]).reset_index(drop=True).drop_duplicates()
extended.to_csv('no_errors_appended.csv')

## Testing Zone

In [24]:
"""
url = 'https://projects.propublica.org/nonprofits/organizations/263405689'
html = scrape_website(url)
soup = BeautifulSoup(html.text)
links = soup.find_all('a', class_='action xml')
base_url = 'https://projects.propublica.org'
urls = [base_url + x.get('href') for x in links if x.text == '990']
"""

In [25]:
"""
test = [x for x in soup.find_all(class_='action xml') if re.search(r'990\b', x.text)]
test2 = [base_url + x.get('href') if x.name == 'a' else base_url + x.select_one('select.action.xml option[data-href]').get('data-href') for x in test]
"""

## Working with our 990 information

In [65]:
info990

NameError: name 'info990' is not defined

In [27]:
# clean up data
extended = pd.concat([pd.read_csv('no_errors.csv'), no_error]).reset_index(drop=True).drop_duplicates()

In [28]:
info = pd.read_csv('no_errors.csv')

In [29]:
info = info.set_index('EIN_YEAR')

In [30]:
info

,ba,EIN,Tax Year,Tax End Date,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total
EIN_YEAR,,,,,,,,,,,,,
952794253_2021,North County Lifeline Inc\ndba Lifeline Commun...,952794253,2021,06-30-2022,92084,0,0,0,0,13405575,451714,223226,13857289
952794253_2020,North County Lifeline Inc\ndba Lifeline Commun...,952794253,2020,06-30-2021,92056,0,0,0,0,13098737,679469,189639,13778206
952794253_2019,North County Lifeline Inc,952794253,2019,06-30-2020,92056,0,0,0,0,12446965,392396,251569,12839361
952794253_2018,North County Lifeline Inc,952794253,2018,06-30-2019,92056,0,0,0,0,10496439,353245,235480,10849684
952794253_2017,North County Lifeline Inc,952794253,2017,06-30-2018,92056,0,0,0,0,9738384,287018,207107,10025402
...,...,...,...,...,...,...,...,...,...,...,...,...,...
237332048_2018,THE SAN DIEGO LESBIANGAY BISEXUAL\nTRANSGENDER...,237332048,2018,06-30-2019,92163,0,0,0,0,2417854,4898662,0,7316516
237332048_2017,THE SAN DIEGO LESBIANGAY BISEXUAL\nTRANSGENDER...,237332048,2017,06-30-2018,92163,0,0,0,0,2618909,4786412,0,7405321
237332048_2016,THE SAN DIEGO LESBIANGAY BISEXUAL\nTRANSGENDER...,237332048,2016,06-30-2017,92163,0,0,0,0,2674347,4105894,0,6780241


Find errors in outputted dataset, this will output the ***EIN_YEAR*** values that are outputted incorrectly. The error seems to always be in the ***Fundraising Events*** column

In [ ]:
info_EINerrs = find_errors(no_error)
info_EINerrs

In [32]:
# EIN_YEARs with errors
info_errs = info[info.index.isin(info_EINerrs)]

In [33]:
# grabbing EINs that had an error in an EIN_YEAR
EINerrs = [int(err[:-5]) for (err) in info_EINerrs]
EINerrs

[951874073,
 272917644,
 952039198,
 952039198,
 953302967,
 571214920,
 953368020,
 331146733,
 263405689,
 953497926,
 550806460]

Correct errors for the ***Fundraising Events*** by using `grabber(errors = True)`

In [36]:
filtered_info_errs = grabber(EINerrs, verbose=True, errors=True)

0 / 11
0 https://projects.propublica.org/nonprofits/download-xml?object_id=202310589349301711
1 https://projects.propublica.org/nonprofits/download-xml?object_id=202240459349300229
2 https://projects.propublica.org/nonprofits/download-xml?object_id=202100349349301560
3 https://projects.propublica.org/nonprofits/download-xml?object_id=202010449349300941
4 https://projects.propublica.org/nonprofits/download-xml?object_id=201920429349301232
5 https://projects.propublica.org/nonprofits/download-xml?object_id=201840519349301264
6 https://projects.propublica.org/nonprofits/download-xml?object_id=201700879349300610
7 https://projects.propublica.org/nonprofits/download-xml?object_id=201612219349301556
8 https://projects.propublica.org/nonprofits/download-xml?object_id=201531339349305568
9 https://projects.propublica.org/nonprofits/download-xml?object_id=201421219349301317
10 https://projects.propublica.org/nonprofits/download-xml?object_id=201310469349302701
11 https://projects.propublica.org/

93 https://projects.propublica.org/nonprofits/download-xml?object_id=201921359349305402
94 https://projects.propublica.org/nonprofits/download-xml?object_id=201821349349310442
95 https://projects.propublica.org/nonprofits/download-xml?object_id=201741149349300624
96 https://projects.propublica.org/nonprofits/download-xml?object_id=201601349349301395
97 https://projects.propublica.org/nonprofits/download-xml?object_id=201511289349301396
98 https://projects.propublica.org/nonprofits/download-xml?object_id=201441359349305494
99 https://projects.propublica.org/nonprofits/download-xml?object_id=201222059349300532
100 https://projects.propublica.org/nonprofits/download-xml?object_id=201132989349300713
10 / 11
101 https://projects.propublica.org/nonprofits/download-xml?object_id=202320899349301857
102 https://projects.propublica.org/nonprofits/download-xml?object_id=202221389349301022
103 https://projects.propublica.org/nonprofits/download-xml?object_id=202110749349300046
104 https://projects

In [37]:
filtered_info_errs

,ba,EIN,Tax Year,Tax End Date,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total
0,SAN DIEGO RESCUE MISSION INC,951874073,2021.0,09-30-2022,92138,0,0,155764,0,0,26121296,8271604,26277060
1,SAN DIEGO RESCUE MISSION INC,951874073,2020.0,09-30-2021,92138,0,0,142855,0,0,21978558,10039786,22121413
2,SAN DIEGO RESCUE MISSION INC,951874073,2019.0,09-30-2020,92138,0,0,50825,0,0,16540608,7454894,16591433
3,SAN DIEGO RESCUE MISSION INC,951874073,2018.0,09-30-2019,92138,0,0,297819,0,0,16921789,8637829,17219608
4,SAN DIEGO RESCUE MISSION INC,951874073,2017.0,09-30-2018,92138,0,0,0,0,0,17095088,9929227,17095088
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,Bilateral Safety Corridor Coalition,550806460,2019.0,12-31-2019,91950,0,0,0,0,0,269047,0,269047
105,BILATERAL SAFETY CORRIDOR COALITION,550806460,2018.0,12-31-2018,91950,0,0,0,0,0,297479,0,297479
106,Bilateral Safety Corridor Coalition,550806460,2017.0,12-31-2017,91950,0,0,0,0,171066,159109,0,330175
107,Bilateral Safety Corridor Coalition,550806460,2016.0,12-31-2016,91950,0,0,0,0,188786,236171,0,424957


Drop NaN values, create an *EIN_YEAR* column, and set it as index

In [38]:
info_no_errors = filtered_info_errs.dropna().reset_index(drop=True)
int_columns = info_no_errors.columns.difference(['ba','Tax End Date'])  # Skip the 'Tax End Date' column
info_no_errors[int_columns] = info_no_errors[int_columns].astype(int)
info_no_errors['EIN_YEAR'] = info_no_errors['EIN'].astype(str) + '_' + info_no_errors['Tax Year'].astype(str)
info_no_errors = info_no_errors.set_index('EIN_YEAR')
info_no_errors

,ba,EIN,Tax Year,Tax End Date,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total
EIN_YEAR,,,,,,,,,,,,,
951874073_2021,SAN DIEGO RESCUE MISSION INC,951874073,2021,09-30-2022,92138,0,0,155764,0,0,26121296,8271604,26277060
951874073_2020,SAN DIEGO RESCUE MISSION INC,951874073,2020,09-30-2021,92138,0,0,142855,0,0,21978558,10039786,22121413
951874073_2019,SAN DIEGO RESCUE MISSION INC,951874073,2019,09-30-2020,92138,0,0,50825,0,0,16540608,7454894,16591433
951874073_2018,SAN DIEGO RESCUE MISSION INC,951874073,2018,09-30-2019,92138,0,0,297819,0,0,16921789,8637829,17219608
951874073_2017,SAN DIEGO RESCUE MISSION INC,951874073,2017,09-30-2018,92138,0,0,0,0,0,17095088,9929227,17095088
...,...,...,...,...,...,...,...,...,...,...,...,...,...
550806460_2019,Bilateral Safety Corridor Coalition,550806460,2019,12-31-2019,91950,0,0,0,0,0,269047,0,269047
550806460_2018,BILATERAL SAFETY CORRIDOR COALITION,550806460,2018,12-31-2018,91950,0,0,0,0,0,297479,0,297479
550806460_2017,Bilateral Safety Corridor Coalition,550806460,2017,12-31-2017,91950,0,0,0,0,171066,159109,0,330175


In [39]:
find_errors(info_no_errors)

('951874073_2015',)

The EIN_YEAR '951874073_2015' has an error we found manually so we're just going to set the 'Fundraising Events' value to 0

In [43]:
info_no_errors.loc['951874073_2015', 'Fundraising Events'] = 0
find_errors(info_no_errors)

()

In [44]:
info_no_errors.to_csv('info_no_errors.csv')

## EIN Dataset

In [4]:
info = pd.read_csv('no_errors.csv')
info_no_errors = pd.read_csv('info_no_errors.csv')

Replace errors with corrected data in    *info_no_errors*    by using `replace_corrected_data_and_standardize`

In [18]:
info_copy = info.copy()
info_no_errors_copy = info_no_errors.copy()

info_copy.rename(columns={'ba': 'Organization Name'}, inplace=True)
info_no_errors_copy.rename(columns={'ba': 'Organization Name'}, inplace=True)

EIN_dataset_df = replace_corrected_data_and_standardize(info_copy, info_no_errors_copy)

EIN_dataset_df.set_index('EIN_YEAR', inplace=True)
EIN_dataset_df = EIN_dataset_df.drop('index',axis=1)
EIN_dataset_df

,Organization Name,EIN,Tax Year,Tax End Date,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total,Philanthropic_Total
EIN_YEAR,,,,,,,,,,,,,,
952794253_2021,NORTH COUNTY LIFELINE INC DBA LIFELINE COMMUNI...,952794253,2021,06-30-2022,92084,0,0,0,0,13405575,451714,223226,13857289,451714
952794253_2020,NORTH COUNTY LIFELINE INC DBA LIFELINE COMMUNI...,952794253,2020,06-30-2021,92056,0,0,0,0,13098737,679469,189639,13778206,679469
952794253_2019,NORTH COUNTY LIFELINE INC,952794253,2019,06-30-2020,92056,0,0,0,0,12446965,392396,251569,12839361,392396
952794253_2018,NORTH COUNTY LIFELINE INC,952794253,2018,06-30-2019,92056,0,0,0,0,10496439,353245,235480,10849684,353245
952794253_2017,NORTH COUNTY LIFELINE INC,952794253,2017,06-30-2018,92056,0,0,0,0,9738384,287018,207107,10025402,287018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237332048_2018,THE SAN DIEGO LESBIANGAY BISEXUAL TRANSGENDER ...,237332048,2018,06-30-2019,92163,0,0,0,0,2417854,4898662,0,7316516,4898662
237332048_2017,THE SAN DIEGO LESBIANGAY BISEXUAL TRANSGENDER ...,237332048,2017,06-30-2018,92163,0,0,0,0,2618909,4786412,0,7405321,4786412
237332048_2016,THE SAN DIEGO LESBIANGAY BISEXUAL TRANSGENDER ...,237332048,2016,06-30-2017,92163,0,0,0,0,2674347,4105894,0,6780241,4105894


In [61]:
EIN_dataset_df.to_csv('EIN_dataset.csv')

## Getting Manual Pull Rows that aren't included in the Scraped Data

Not all 990 years have an XML file attached for the earlier years, but they were recorded in the manual pull so we can access those years from the manual pull and merge that data into our current dataset

In [47]:
# Manually inputted Tax End Year for the eins in 'pull_these' into the downloaded csv
raw_manual_pull = pd.read_csv('990 Manual Pull - Tax End Date.csv')
raw_manual_pull.set_index('EIN_YEAR',inplace=True)
raw_manual_pull

,ba,EIN,Tax Year,Tax End Date,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,"All Other Contributions gifts, grants and similar amounts not included above",Noncash constributions included in lines 1a - 1f,Total,Type,Unnamed: 14,Unnamed: 15,Unnamed: 16
EIN_YEAR,,,,,,,,,,,,,,,,,
113723093_2015,Regional Task Force on the Homeless,113723093.0,2015.0,NaN,92123.0,0,0,0,0,1110060,47330,0,1157390,A,1157390.0,True,0.0
113723093_2016,Regional Task Force on the Homeless,113723093.0,2016.0,NaN,92123.0,0,6677,0,0,1922156,65214,0,1994047,A,1994047.0,True,0.0
113723093_2018,Regional Task Force on the Homeless,113723093.0,2018.0,NaN,92123.0,0,0,0,0,2505451,865020,0,3370471,A,3370471.0,True,0.0
113723093_2019,Regional Task Force on the Homeless,113723093.0,2019.0,NaN,92123.0,0,0,0,0,9344677,182239,0,9526916,A,9526916.0,True,0.0
113723093_2020,Regional Task Force on the Homeless,113723093.0,2020.0,NaN,92123.0,0,0,0,0,17841834,183535,0,18025369,A,18025369.0,True,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571214920_2020,Operation Hope North County,571214920.0,2020.0,NaN,92084.0,0,0,0,0,422615,692114,0,1114729,NaN,1114729.0,True,NaN
571162424_2021,Kingdom Causes dba City Net,571162424.0,2021.0,NaN,90807.0,89250,0,0,0,429733,790394,27870,1309377,NaN,1309377.0,True,NaN
NaN,NaN,NaN,NaN,NaN,NaN,"$35,571,906.00","$47,967.00","$19,229,327.00","$23,349,134.00","$3,384,170,346.00","$759,399,657.00","$218,330,487.00","$4,230,190,126.00",NaN,NaN,NaN,NaN


In [64]:
# drop rows that have NaN EIN values
manual_pull = raw_manual_pull[raw_manual_pull.get('Type') == 'A'].dropna(thresh=1).dropna(subset=['EIN']).drop(['Type'], axis=1)
manual_pull

,ba,EIN,Tax Year,Tax End Date,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,"All Other Contributions gifts, grants and similar amounts not included above",Noncash constributions included in lines 1a - 1f,Total,Unnamed: 14,Unnamed: 15,Unnamed: 16
EIN_YEAR,,,,,,,,,,,,,,,,
113723093_2015,Regional Task Force on the Homeless,113723093.0,2015.0,NaN,92123.0,0,0,0,0,1110060,47330,0,1157390,1157390.0,True,0.0
113723093_2016,Regional Task Force on the Homeless,113723093.0,2016.0,NaN,92123.0,0,6677,0,0,1922156,65214,0,1994047,1994047.0,True,0.0
113723093_2018,Regional Task Force on the Homeless,113723093.0,2018.0,NaN,92123.0,0,0,0,0,2505451,865020,0,3370471,3370471.0,True,0.0
113723093_2019,Regional Task Force on the Homeless,113723093.0,2019.0,NaN,92123.0,0,0,0,0,9344677,182239,0,9526916,9526916.0,True,0.0
113723093_2020,Regional Task Force on the Homeless,113723093.0,2020.0,NaN,92123.0,0,0,0,0,17841834,183535,0,18025369,18025369.0,True,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270447059_2018,Dreams for Change Inc.,270447059.0,2018.0,NaN,92176.0,0,0,0,0,345041,492397,0,837438,837438.0,True,NaN
270447059_2019,Dreams for Change Inc.,270447059.0,2019.0,NaN,92176.0,0,0,0,0,600504,421536,0,1022040,1022040.0,True,NaN
270447059_2020,Dreams for Change Inc.,270447059.0,2020.0,NaN,92176.0,0,0,0,0,717584,434903,0,1152487,1152487.0,True,NaN


*EIN_YEAR*s that are in the manual pull but weren't pulled by XML and need to be added to our dataset

In [49]:
pull_these = list(manual_pull.index.difference(EIN_dataset_df.index))
pull_these

['330006089_2015',
 '330433314_2015',
 '330433314_2016',
 '330433314_2017',
 '330433314_2018',
 '330602842_2015',
 '330602842_2016',
 '330602842_2017',
 '330602842_2018',
 '330602842_2019',
 '465055513_2016',
 '465055513_2017',
 '562613191_2015',
 '562613191_2016',
 '941676390_2016',
 '941676390_2017',
 '941676390_2018',
 '941676390_2019',
 '952111196_2015',
 '952850121_2015',
 '952850121_2016']

In [50]:
to_pull = np.unique([ein[:-5] for ein in pull_these])
to_pull

array(['330006089', '330433314', '330602842', '465055513', '562613191',
       '941676390', '952111196', '952850121'], dtype='<U9')

Fill ***not_scraped*** with the manual pull data for the ***pull_these*** EIN values

In [51]:
not_scraped = manual_pull.loc[pull_these]
not_scraped['EIN'] = [int(ein) for ein in not_scraped['EIN']]
not_scraped['Tax Year'] = [int(year) for year in not_scraped['Tax Year']]
not_scraped['Location (Zipcode)'] = [int(loc) for loc in not_scraped['Location (Zipcode)']]
not_scraped = not_scraped.drop(['Unnamed: 14','Unnamed: 15','Unnamed: 16'],axis=1).reset_index()
not_scraped

,EIN_YEAR,ba,EIN,Tax Year,Tax End Date,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,"All Other Contributions gifts, grants and similar amounts not included above",Noncash constributions included in lines 1a - 1f,Total
0,330006089_2015,Deaf Community Services of San Diego,330006089,2015,6-30-2016,92108,3270,0,0,0,1329774,61532,0,1394576
1,330433314_2015,National Conflict Resolution Center,330433314,2015,6-30-2016,92101,0,0,0,0,851714,1076360,0,1928074
2,330433314_2016,National Conflict Resolution Center,330433314,2016,6-30-2017,92101,0,0,0,0,806892,1300903,0,2107795
3,330433314_2017,National Conflict Resolution Center,330433314,2017,6-30-2018,92101,0,0,0,0,1514007,1231820,0,2745827
4,330433314_2018,National Conflict Resolution Center,330433314,2018,6-30-2019,92101,0,0,0,0,1265617,1689439,0,2955056
5,330602842_2015,Association for Community Housing Solutions,330602842,2015,12-31-2015,92123,0,0,0,0,1237593,454,0,1238047
6,330602842_2016,Association for Community Housing Solutions,330602842,2016,12-31-2016,92123,0,0,0,0,1485908,1212,0,1487120
7,330602842_2017,Association for Community Housing Solutions,330602842,2017,12-31-2017,92123,0,0,0,0,1564629,1120,0,1565749
8,330602842_2018,Association for Community Housing Solutions,330602842,2018,12-31-2018,92123,0,0,0,0,1610341,5416,0,1615757
9,330602842_2019,Association for Community Housing Solutions,330602842,2019,12-31-2019,92123,0,0,0,0,1527348,8561,0,1535909


Rename columns and export

In [52]:
not_scraped.columns = ['EIN_YEAR','Organization Name', 'EIN', 'Tax Year', 'Tax End Date','Location (Zipcode)', 'Federate Campaigns', 'Membership Dues', \
                       'Fundraising Events', 'Related Organizations', 'Government Grants', 'All Other Contributions', \
                       'Noncash Contributions', 'Total']

int_columns = not_scraped.columns.difference(['Organization Name','EIN_YEAR', 'Tax End Date'])  # Skip the 'Tax End Date' column
not_scraped[int_columns] = not_scraped[int_columns].apply(pd.to_numeric, errors='coerce')
not_scraped.set_index('EIN_YEAR', inplace = True)
not_scraped.to_csv('not_scraped.csv')

Add ***not_scraped*** to our ***EIN_dataset_df*** for our full dataset

In [56]:
phil_df = pd.concat([EIN_dataset_df, not_scraped])
phil_df.sort_values(by='EIN_YEAR', inplace=True)
phil_df

,Organization Name,EIN,Tax Year,Tax End Date,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total,Philanthropic_Total
EIN_YEAR,,,,,,,,,,,,,,
113723093_2014,REGIONAL TASK FORCE ON THE HOMELESS INC,113723093,2014,06-30-2015,92123,0,0,0,0,991765,94796,0,1086561,94796.0
113723093_2015,REGIONAL TASK FORCE ON THE HOMELESS,113723093,2015,06-30-2016,92123,0,0,0,0,1110060,47330,0,1157390,47330.0
113723093_2016,REGIONAL TASK FORCE ON THE HOMELESS,113723093,2016,06-30-2017,92123,0,6677,0,0,1922156,65214,0,1994047,71891.0
113723093_2017,REGIONAL TASK FORCE ON THE HOMELESS,113723093,2017,06-30-2018,92123,0,0,0,0,2021667,738875,0,2760542,738875.0
113723093_2018,REGIONAL TASK FORCE ON THE HOMELESS,113723093,2018,06-30-2019,92123,0,0,0,0,2505451,865020,0,3370471,865020.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956379598_2017,CENTER FOR COMMUNITY SOLUTIONS,956379598,2017,06-30-2018,92109,0,0,286098,0,4101800,719296,35080,5107194,1005394.0
956379598_2018,CENTER FOR COMMUNITY SOLUTIONS,956379598,2018,06-30-2019,92109,0,0,431150,0,4702859,731655,33999,5865664,1162805.0
956379598_2019,CENTER FOR COMMUNITY SOLUTIONS,956379598,2019,06-30-2020,92109,0,0,169778,0,4963840,943753,50456,6077371,1113531.0


In [53]:
phil_df.to_csv('Phil Dataset.csv')

## Parsing dataset by month

In [57]:
data = pd.read_csv('Phil Dataset.csv')

Add rows for each month

In [61]:
# Convert 'Tax End Date' column to datetime format
data['Tax End Date'] = pd.to_datetime(data['Tax End Date'])

# Apply the function to each row and expand the DataFrame
expanded_data_fiscal_year = data.apply(distribute_values_fiscal_year, axis=1)
expanded_data_fiscal_year = pd.DataFrame([item for sublist in expanded_data_fiscal_year for item in sublist])

# Save the expanded data to a new CSV file
expanded_data_fiscal_year = expanded_data_fiscal_year.drop_duplicates()
expanded_data_fiscal_year.to_csv('Phil Dataset - Analysis.csv')

,EIN_YEAR,Organization Name,EIN,Tax Year,Tax End Date,Location (Zipcode),Federate Campaigns,Membership Dues,Fundraising Events,Related Organizations,Government Grants,All Other Contributions,Noncash Contributions,Total,Philanthropic_Total
0,113723093_2014,REGIONAL TASK FORCE ON THE HOMELESS INC,113723093,2014,07-31-2014,92123,0,0.0,0.000000,0.0,82647.083333,7899.666667,0.000000,90546.75,7899.666667
0,113723093_2014,REGIONAL TASK FORCE ON THE HOMELESS INC,113723093,2014,08-31-2014,92123,0,0.0,0.000000,0.0,82647.083333,7899.666667,0.000000,90546.75,7899.666667
0,113723093_2014,REGIONAL TASK FORCE ON THE HOMELESS INC,113723093,2014,09-30-2014,92123,0,0.0,0.000000,0.0,82647.083333,7899.666667,0.000000,90546.75,7899.666667
0,113723093_2014,REGIONAL TASK FORCE ON THE HOMELESS INC,113723093,2014,10-31-2014,92123,0,0.0,0.000000,0.0,82647.083333,7899.666667,0.000000,90546.75,7899.666667
0,113723093_2014,REGIONAL TASK FORCE ON THE HOMELESS INC,113723093,2014,11-30-2014,92123,0,0.0,0.000000,0.0,82647.083333,7899.666667,0.000000,90546.75,7899.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,956379598_2021,CENTER FOR COMMUNITY SOLUTIONS,956379598,2021,02-28-2022,92109,0,0.0,38652.583333,0.0,459745.500000,70906.416667,1354.666667,569304.50,109559.000000
641,956379598_2021,CENTER FOR COMMUNITY SOLUTIONS,956379598,2021,03-31-2022,92109,0,0.0,38652.583333,0.0,459745.500000,70906.416667,1354.666667,569304.50,109559.000000
641,956379598_2021,CENTER FOR COMMUNITY SOLUTIONS,956379598,2021,04-30-2022,92109,0,0.0,38652.583333,0.0,459745.500000,70906.416667,1354.666667,569304.50,109559.000000
641,956379598_2021,CENTER FOR COMMUNITY SOLUTIONS,956379598,2021,05-31-2022,92109,0,0.0,38652.583333,0.0,459745.500000,70906.416667,1354.666667,569304.50,109559.000000
